In [9]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau


In [8]:
!pip install keras-tuner


   ---------------------------------------- 0.0/129.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/129.1 kB ? eta -:--:--
   ------------ -------------------------- 41.0/129.1 kB 960.0 kB/s eta 0:00:01
   ---------------------------------------- 129.1/129.1 kB 1.9 MB/s eta 0:00:00


In [10]:
# Set the local paths for the datasets
Eczema_dir = r'Skin_disease_data\1. Eczema'
Psoriasis_pictures_Lichen_Planus_and_related_diseases_dir = r'Skin_disease_data\7. Psoriasis'


In [11]:
filepaths = []
labels = []
dict_list = [Eczema_dir, Psoriasis_pictures_Lichen_Planus_and_related_diseases_dir]
class_labels = ['Eczema', 'Psoriasis pictures Lichen Planus and related diseases']

for i, j in enumerate(dict_list):
    flist = os.listdir(j)
    for f in flist:
        fpath = os.path.join(j, f)
        filepaths.append(fpath)
        labels.append(class_labels[i])

Fseries = pd.Series(filepaths, name="filepaths")
Lseries = pd.Series(labels, name="labels")
skin_data = pd.concat([Fseries, Lseries], axis=1)
skin_df = pd.DataFrame(skin_data)
print(skin_df.head())
print(skin_df["labels"].value_counts())

                              filepaths  labels
0   Skin_disease_data\1. Eczema\0_0.jpg  Eczema
1   Skin_disease_data\1. Eczema\0_1.jpg  Eczema
2  Skin_disease_data\1. Eczema\0_10.jpg  Eczema
3  Skin_disease_data\1. Eczema\0_11.jpg  Eczema
4  Skin_disease_data\1. Eczema\0_12.jpg  Eczema
labels
Psoriasis pictures Lichen Planus and related diseases    2055
Eczema                                                   1677
Name: count, dtype: int64


In [12]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from kerastuner import HyperModel, RandomSearch

def build_model(hp):
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False  # Freeze the base model

    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())

    # Tune the number of units in the Dense layer
    hp_units = hp.Int('units', min_value=128, max_value=1024, step=128)
    model.add(Dense(units=hp_units, activation='relu'))
    
    # Tune the dropout rate
    hp_dropout = hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)
    model.add(Dropout(rate=hp_dropout))
    
    model.add(Dense(2, activation='softmax'))

    # Tune the learning rate for the optimizer
    hp_learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
    model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model



C:\Users\SHARUK\AppData\Local\Temp\ipykernel_19168\3814032278.py:6: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperModel, RandomSearch


In [13]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=3,
    directory='my_dir',
    project_name='inception_tuning'
)


Reloading Tuner from my_dir\inception_tuning\tuner0.json


In [15]:
# Assume train, val, and test ImageDataGenerators are defined as in previous examples
train_images, test_images = train_test_split(skin_df, test_size=0.2, random_state=42)
train_set, val_set = train_test_split(train_images, test_size=0.2, random_state=42)

train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.inception_v3.preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

image_gen = ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_v3.preprocess_input)

train = train_datagen.flow_from_dataframe(
    dataframe=train_set, x_col="filepaths", y_col="labels",
    target_size=(224, 224),
    color_mode='rgb',
    class_mode="categorical",
    batch_size=32,
    shuffle=True
)

val = image_gen.flow_from_dataframe(
    dataframe=val_set, x_col="filepaths", y_col="labels",
    target_size=(224, 224),
    color_mode='rgb',
    class_mode="categorical",
    batch_size=32,
    shuffle=False
)






Found 2388 validated image filenames belonging to 2 classes.
Found 597 validated image filenames belonging to 2 classes.


In [16]:
tuner.search(train, epochs=10, validation_data=val)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
The optimal number of units in the dense layer is {best_hps.get('units')} and
the optimal dropout rate is {best_hps.get('dropout')}.
The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.
""")


Trial 10 Complete [01h 15m 41s]
val_accuracy: 0.7191513180732727

Best val_accuracy So Far: 0.7191513180732727
Total elapsed time: 1d 03h 33m 38s

The optimal number of units in the dense layer is 640 and
the optimal dropout rate is 0.2.
The optimal learning rate for the optimizer is 0.0002479277711879142.



In [20]:
test = image_gen.flow_from_dataframe(
    dataframe=test_images, x_col="filepaths", y_col="labels",
    target_size=(224, 224),
    color_mode='rgb',
    class_mode="categorical",
    batch_size=32,
    shuffle=False
)

Found 747 validated image filenames belonging to 2 classes.
